**Pytorch Function Regression Example**

In [ ]:
# CPU-only PyTorch (no CUDA needed)
!pip -q install --upgrade pip
!pip -q install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cpu
!pip -q install tqdm

**Importing Packages**

This is the convention of torch packages to import.  It is recommended to print whether "cuda" is available and define the "device" for your script.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from tqdm import tqdm
import sys, random
import matplotlib.pyplot as plt

print("Python:", sys.version.split()[0])
print("torch :", torch.__version__)
print("torchvision:", __import__("torchvision").__version__)
print("torchaudio :", __import__("torchaudio").__version__)
print("CUDA available?", torch.cuda.is_available())

device = torch.device("cpu") # gpu device = "cuda"
print(f'{device=}')

torch.manual_seed(0); random.seed(0); np.random.seed(0)
torch.set_printoptions(precision=3)

**Load Data**

Loading data, splitting into training, validation, and testing (if applicable) splits.  Must be turned into torch.tensor at some point.

In [ ]:
## LOAD DATA

def load_data():
  x = np.linspace(-5, 5, 1001)
  y = np.where(x < 2.5,
             np.sin(np.exp(0.8*x)) * np.exp(-0.3*x),
             (-1/2.5)*x + 1.5) + np.random.randn(len(x)) * 0.1
  y_fun = np.where(x < 2.5,
             np.sin(np.exp(0.8*x)) * np.exp(-0.3*x),
             (-1/2.5)*x + 1.5)
  # validation split is (-3, -2.5) and (4, 4.5), everythin else is training split
  indices = list(range(len(x)))
  train_indices = indices[:200] + indices[250:900] + indices[950:]
  val_indices = indices[200:250] + indices[900:950]
  test_indices = indices[750:900]
  x_train = x[train_indices]
  y_train = y[train_indices]
  x_val = x[val_indices]
  y_val = y[val_indices]
  return x_train, y_train, x_val, y_val


In [ ]:
x_train, y_train, x_val, y_val = load_data()

x = np.linspace(-5, 5, 1001)
y_fun = np.where(x < 2.5,
             np.sin(np.exp(0.8*x)) * np.exp(-0.3*x),
             (-1/2.5)*x + 1.5)
plt.scatter(x_train, y_train, color='green', s=4, label='Training data')
plt.scatter(x_val, y_val, color='blue', s=4, label='Validation data')
plt.plot(x, y_fun, color='black', linewidth=2, label='True function') # black
plt.legend()
plt.show()

**Dataset Class**

Understanding the dataset class is crucial to train models in bespoke applications.  This class only requires the __len and __getitem methods.  This class takes your input and output and zips them together for your dataloader.  While there is an importable module for this (it is just the following code), keeping this section in your script allows more complex operations on your input and output.  Within getitem, you can modify your data on the fly if it is undesirable to load it initially.

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, inputs,  targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        inputs = self.inputs[idx]
        target = self.targets[idx]
        return inputs, target

**Dataloaders**

Dataloaders take your dataset as an input, batch it, and return batches of your input, output pairs as singular tensors.

In [ ]:
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).float()
x_val = torch.from_numpy(x_val).float()
y_val = torch.from_numpy(y_val).float()

train_dataset = CustomDataset(x_train, y_train)
val_dataset = CustomDataset(x_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=True, drop_last=True)

In [ ]:
print(train_dataset[100])
for input, target in train_loader:
  print(input, target)
  break

**Training Loop**

In [ ]:
def train_epoch(dataloader, model, optimizer, loss_function, device):
    model.train()
    total_loss = 0
    for inputs, targets in dataloader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        predictions = model(inputs)
        loss = loss_function(predictions, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)


def validate_epoch(dataloader, model, loss_function, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            predictions = model(inputs)
            loss = 0
            loss = loss_function(predictions, targets)
            total_loss += loss.item()
    return total_loss / len(dataloader)

def train_and_validate(epochs, model, optimizer, loss_function, train_loader, val_loader, device):

    model.to(device)  # Move model to the appropriate device
    for epoch in range(epochs):
        # print(f"Epoch {epoch + 1}\n-------------------------------")
        train_loss = train_epoch(train_loader, model, optimizer, loss_function, device)
        val_loss = validate_epoch(val_loader, model, loss_function, device)
        print(f"Epoch {epoch + 1} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

**Model Definition**

In [ ]:
class ModelExample(nn.Module):
    def __init__(self):
        super(ModelExample, self).__init__()

        # Define deeper convolutional layers
        self.lin1 = nn.Linear(1, 16)
        self.lin2 = nn.Linear(16,16)
        self.lin3 = nn.Linear(16,16)
        self.lin4 = nn.Linear(16,1)
        self.sig = nn.ReLU()

    def forward(self, x):
        x = self.lin1(x)
        x = self.sig(x)
        x = self.lin2(x)
        x = self.sig(x)
        x = self.lin3(x)
        x = self.sig(x)
        x = self.lin4(x)
        x = self.sig(x)
        return x

**Define Instance of Model, the optimizer, and the loss function**

In [ ]:
# Define Model Instance
model = ModelExample()

# Load Optimizer and Loss Function
optimizer = optim.Adam(model.parameters(), lr=0.1)
loss_function = nn.MSELoss()

In [ ]:
train_and_validate(30, model, optimizer, loss_function, train_loader, val_loader, device)

In [ ]:
class ModelExample(nn.Module):
    def __init__(self, embed_dim):
        super(ModelExample, self).__init__()

        # Define deeper convolutional layers
        self.lin1 = nn.Linear(1, embed_dim)
        self.lin2 = nn.Linear(embed_dim, embed_dim)
        self.lin3 = nn.Linear(embed_dim,embed_dim)
        self.lin4 = nn.Linear(embed_dim,embed_dim)
        self.lin5 = nn.Linear(embed_dim,1)
        self.sig = nn.ReLU() # nn.Sigmoid()

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.lin1(x)
        x = self.sig(x)
        x = self.lin2(x)
        x = self.sig(x)
        x = self.lin3(x)
        x = self.sig(x)
        x = self.lin4(x)
        x = self.sig(x)
        x = self.lin5(x)
        return x.squeeze()

# Define Model Instance
model = ModelExample(15)

# Count model Parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_params}")

# Load Optimizer and Loss Function
optimizer = optim.Adam(model.parameters(), lr=0.0012)
loss_function = nn.MSELoss()

train_and_validate(30, model, optimizer, loss_function, train_loader, val_loader, device)

In [ ]:
x = np.linspace(-5, 5, 1001)
y_pred = model(torch.Tensor(x).unsqueeze(1))
y_pred = y_pred.detach().numpy()
y_fun = np.where(x < 2.5,
             np.sin(np.exp(0.8*x)) * np.exp(-0.3*x),
             (-1/2.5)*x + 1.5)
plt.scatter(x, y_pred, color='green', s=4, label='Model Predictions')
plt.plot(x, y_fun, color='black', linewidth=2, label='True function') # black
# plot vertical lines at -3, -2.5, 4, 4.5
plt.axvline(x=-3, color='red', linestyle='--')
plt.axvline(x=-2.5, color='red', linestyle='--')
plt.axvline(x=4, color='red', linestyle='--')
plt.axvline(x=4.5, color='red', linestyle='--')
plt.legend()
plt.show()

In [ ]:
## LOAD DATA
# our function is y = sin(e^(0.8x))e^(-0.3x) + noise
def load_data_50():
  x = np.linspace(-5, 5, 51)
  y = np.where(x < 2.5,
             np.sin(np.exp(0.8*x)) * np.exp(-0.3*x),
             (-1/2.5)*x + 1.5) + np.random.randn(len(x)) * 0.1
  y_fun = np.where(x < 2.5,
             np.sin(np.exp(0.8*x)) * np.exp(-0.3*x),
             (-1/2.5)*x + 1.5)
  # validation split is (-3, -2.5) and (4, 4.5), everythin else is training split
  indices = list(range(len(x)))
  train_indices = indices[:10] + indices[13:45] + indices[48:]
  val_indices = indices[10:13] + indices[45:48]
  x_train = x[train_indices]
  y_train = y[train_indices]
  x_val = x[val_indices]
  y_val = y[val_indices]
  return x_train, y_train, x_val, y_val

x_train, y_train, x_val, y_val = load_data_50()

x = np.linspace(-5, 5, 1001)
# y_fun = np.sin(np.exp(0.8*x)) * np.exp(-0.3 * x)
y_fun = np.where(x < 2.5,
             np.sin(np.exp(0.8*x)) * np.exp(-0.3*x),
             (-1/2.5)*x + 1.5)
plt.scatter(x_train, y_train, color='green', s=4, label='Training data')
plt.scatter(x_val, y_val, color='blue', s=4, label='Validation data')
plt.plot(x, y_fun, color='black', linewidth=2, label='True function') # black
plt.legend()
plt.show()

In [ ]:
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).float()
x_val = torch.from_numpy(x_val).float()
y_val = torch.from_numpy(y_val).float()

train_dataset = CustomDataset(x_train, y_train)
val_dataset = CustomDataset(x_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True, drop_last=True)

In [ ]:


# Define Model Instance
model = ModelExample(100)

# Count model Parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_params}")

# Load Optimizer and Loss Function
optimizer = optim.Adam(model.parameters(), lr=0.0012)
loss_function = nn.MSELoss()

train_and_validate(400, model, optimizer, loss_function, train_loader, val_loader, device)

In [ ]:
x = np.linspace(-5, 5, 1001)
y_pred = model(torch.Tensor(x).unsqueeze(1))
y_pred = y_pred.detach().numpy()
y_fun = np.where(x < 2.5,
             np.sin(np.exp(0.8*x)) * np.exp(-0.3*x),
             (-1/2.5)*x + 1.5)
plt.scatter(x, y_pred, color='green', s=4, label='Model Predictions')
plt.scatter(x_train, y_train, color='blue', s=4, label='Data')
plt.plot(x, y_fun, color='black', linewidth=2, label='True function') # black
# plot vertical lines at -3, -2.5, 4, 4.5
plt.axvline(x=-3, color='red', linestyle='--')
plt.axvline(x=-2.5, color='red', linestyle='--')
plt.axvline(x=4, color='red', linestyle='--')
plt.axvline(x=4.5, color='red', linestyle='--')
plt.legend()
plt.show()

**Custom Loss Function**

You can define a custom loss function as a normal function, or you can use the nn.Module if you need class-based functionality.  In the latter case, your function itself goes under "forward".

In [ ]:
# Run with custom loss function.  If prediction is above our data up to 0.1 above, we do not penalize
# We can do this with a normal function, or you can use the nn.Module if you need class-based functionality
def custom_loss(y_pred, y_true):
    x = y_pred - y_true
    loss = (torch.clamp(-x, min=0))**2 + (torch.clamp(x - 0.2, min=0))**2
    return torch.mean(loss)

x_train, y_train, x_val, y_val = load_data()

x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).float()
x_val = torch.from_numpy(x_val).float()
y_val = torch.from_numpy(y_val).float()

train_dataset = CustomDataset(x_train, y_train)
val_dataset = CustomDataset(x_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True, drop_last=True)


# Define Model Instance
model = ModelExample(15)

# Count model Parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_params}")

# Load Optimizer and Loss Function
optimizer = optim.Adam(model.parameters(), lr=0.0012)
loss_function = custom_loss

train_and_validate(30, model, optimizer, loss_function, train_loader, val_loader, device)

In [ ]:
x = np.linspace(-5, 5, 1001)
y_pred = model(torch.Tensor(x).unsqueeze(1))
y_pred = y_pred.detach().numpy()
y_fun = np.where(x < 2.5,
             np.sin(np.exp(0.8*x)) * np.exp(-0.3*x),
             (-1/2.5)*x + 1.5)
plt.scatter(x, y_pred, color='green', s=4, label='Model Predictions')
plt.scatter(x_train, y_train, color='blue', s=4, label='Data')
plt.scatter(x_val, y_val, color='red', s=4, label='Data')
plt.plot(x, y_fun, color='black', linewidth=2, label='True function') # black
# plot vertical lines at -3, -2.5, 4, 4.5
plt.axvline(x=-3, color='red', linestyle='--')
plt.axvline(x=-2.5, color='red', linestyle='--')
plt.axvline(x=4, color='red', linestyle='--')
plt.axvline(x=4.5, color='red', linestyle='--')
plt.legend()
plt.show()

**Other Functionality**



1.   Print the gradients
2.   Freeze certain layers of the model
3.   Save and load checkpoints

